# 基于 GraphSage 的无监督学习


GraphScope提供了处理学习任务的功能。本次教程，我们将会展示GraphScope如何使用GraphSage算法训练一个无监督学习模型。

本次教程的学习任务是链接预测，通过计算在图中顶点之间存在边的概率来预测链接。

在这一任务中，我们使用GraphScope内置的GraphSage算法在 [PPI](https://humgenomics.biomedcentral.com/articles/10.1186/1479-7364-3-3-291) 数据集上训练一个模型，这一训练模型可以用来预测蛋白质结构之间的链接。这一任务可以被看作在一个异构链接网络上的无监督训练任务。

在这一任务中，GraphSage算法会将图中的结构信息和属性信息压缩为每个节点上的低维嵌入向量，这些嵌入和表征可以进一步用来预测节点间的链接。

这一教程将会分为以下几个步骤：
- 启动GraphScope的学习引擎，并将图关联到引擎上
- 使用内置的GraphSAGE模型定义训练过程，并定义相关的超参
- 开始训练


In [ ]:
# Install graphscope package if you are NOT in the Playground

!pip3 install graphscope

In [ ]:
# Import the graphscope module.

import graphscope

graphscope.set_option(show_log=False)  # enable logging

In [ ]:
# Load ppi dataset

from graphscope.dataset import load_ppi

graph = load_ppi()

## Launch learning engine

然后，我们需要定义一个特征列表用于图的训练。训练特征集合必须从点的属性集合中选取。在这个例子中，我们选择了属性集合中所有以"feat-"为前缀的属性作为训练特征集，这一特征集也是PPI数据中点的特征集。

借助定义的特征列表，接下来，我们使用 [graphlearn](https://graphscope.io/docs/reference/session.html#graphscope.Session.graphlearn) 方法来开启一个学习引擎。

在这个例子中，我们在 "graphlearn" 方法中，指定在数据中 "protein" 类型的顶点和 "link' 类型边上进行模型训练。

利用 "gen_labels" 参数，我们将 "protein" 点数据集作为训练集。

In [ ]:
# define the features for learning
paper_features = []
for i in range(50):
    paper_features.append("feat-" + str(i))

# launch a learning engine.
lg = graphscope.graphlearn(
    graph,
    nodes=[("protein", paper_features)],
    edges=[("protein", "link", "protein")],
    gen_labels=[
        ("train", "protein", 100, (0, 100)),
    ],
)

这里我们使用内置的GraphSage模型定义训练过程。在本次示例中，我们使用 tensorflow 作为神经网络后端训练器。

In [ ]:
try:
  # https://www.tensorflow.org/guide/migrate
  import tensorflow.compat.v1 as tf
  tf.disable_v2_behavior()
except ImportError:
  import tensorflow as tf

import argparse
import graphscope.learning.graphlearn.python.nn.tf as tfg
from graphscope.learning.examples import EgoGraphSAGE
from graphscope.learning.examples import EgoSAGEUnsupervisedDataLoader
from graphscope.learning.examples.tf.trainer import LocalTrainer

def parse_args():
  argparser = argparse.ArgumentParser("Train EgoSAGE Unsupervised.")
  argparser.add_argument('--batch_size', type=int, default=512)
  argparser.add_argument('--features_num', type=int, default=50)
  argparser.add_argument('--hidden_dim', type=int, default=128)
  argparser.add_argument('--output_dim', type=int, default=128)
  argparser.add_argument('--nbrs_num', type=list, default=[5, 5])
  argparser.add_argument('--learning_rate', type=float, default=0.01)
  argparser.add_argument('--epochs', type=int, default=2)
  argparser.add_argument('--drop_out', type=float, default=0.0)
  argparser.add_argument('--temperature', type=float, default=0.07)
  argparser.add_argument('--node_type', type=str, default="protein")
  argparser.add_argument('--edge_type', type=str, default="link")
  return argparser.parse_args()
args = parse_args()

# Define Model
dims = [args.features_num] + [args.hidden_dim] * (len(args.nbrs_num) - 1) + [args.output_dim]
model = EgoGraphSAGE(dims, dropout=args.drop_out)

# Prepare train dataset
train_data = EgoSAGEUnsupervisedDataLoader(lg, None, batch_size=args.batch_size,
    node_type=args.node_type, edge_type=args.edge_type, nbrs_num=args.nbrs_num)
src_emb = model.forward(train_data.src_ego)
dst_emb = model.forward(train_data.dst_ego)
neg_dst_emb = model.forward(train_data.neg_dst_ego)
loss = tfg.unsupervised_softmax_cross_entropy_loss(
    src_emb, dst_emb, neg_dst_emb, temperature=args.temperature)
optimizer = tf.train.AdamOptimizer(learning_rate=args.learning_rate)

## 执行训练过程


在定义完训练过程和超参后，现在我们可以使用学习引擎和定义的超参开始训练过程。

In [ ]:
trainer = LocalTrainer()
trainer.train(train_data.iterator, loss, optimizer, epochs=args.epochs)